In [1]:
import networkx as nx
import numpy as np
import math


#dic = {}
#    y = set(G.nodes) - set(list(G.neighbors(X)) + [X])
#    for i in y:
#        dic[i] = len(sorted(nx.common_neighbors(G, X, i)))
#    dic = sorted(dic.items(), key=lambda kv: kv[1], reverse = True)
#    dic = [i[0] for i in dic]

In [2]:
def read_old_edges(file_name):
    f = open(file_name,'r')
    old_edges = []
    for i in f.readlines():
        old_edges.append(i[:-1].split("\t"))
    old_edges = np.array(old_edges)
    return old_edges




def read_new_edges(filename):
    f = open(filename,'r')
    new_edges = []
    for i in f.readlines():
        new_edges.append(i[:-1].split("\t"))
    new_edges = np.array(new_edges)
    return new_edges


def common_friends_number(G,X):
    
    
    z = set(list(G.adj[X]))
    data = [(X,i) for i in G.nodes if i != X]        
    preds = nx.cn_soundarajan_hopcroft(G, data, community = "weight")
    result = []
    for u,v,p in preds:
        result.append([u,v,p])
    result = sorted(result, key = lambda x: x[2], reverse = True)
    result = [i[1] for i in result if i[1] not in list(z)+[X]]
    return result

def jaccard_index(G,X):
    z = set(list(G.adj[X]))
    data = [(X,i) for i in G.nodes if i != X]        
    preds = nx.jaccard_coefficient(G, data)
    result = []
    for u,v,p in preds:
        result.append([u,v,p])
    result = sorted(result, key = lambda x: x[2], reverse = True)
    result = [i[1] for i in result if i[1] not in list(z)+[X]]
    return result


def adamic_adar_index(G,X):
    z = set(list(G.adj[X]))
    data = [(X,i) for i in G.nodes if i != X]        
    preds = nx.adamic_adar_index(G, data)
    result = []
    for u,v,p in preds:
        result.append([u,v,p])
    result = sorted(result, key = lambda x: x[2], reverse = True)
    result = [i[1] for i in result if i[1] not in list(z)+[X]]
    return result


def my_index(G,X):
    dic = {}
    y = set(G.nodes) - set(list(G.neighbors(X)) + [X]) 
    for i in y:
        indexes = len(list(G.neighbors(X))) + len(list(G.neighbors(i)))
        dic[i]  = indexes
    dic = sorted(dic.items(), key=lambda kv: kv[1], reverse = True)
    dic = [i[0] for i in dic]
    return dic

In [3]:
old_edges = read_old_edges('old_edges.txt')
nodes = set(old_edges.flatten())
G = nx.Graph()
G.add_nodes_from(nodes, weight = 1)
G.add_edges_from(old_edges, year = "2010-2016")
new_edges = read_new_edges('new_edges.txt')
X_dic = {}
for i in new_edges.flatten():
    if i not in X_dic:
        X_dic[i] = 1
    else:
        X_dic[i] += 1
X = []
for i in X_dic:
    if X_dic[i] >= 10:
        X.append(i)
index1 = [common_friends_number(G,i)[:10] for i in X]
index2 = [jaccard_index(G,i)[:10] for i in X]
index3 = [adamic_adar_index(G,i)[:10] for i in X]
index4 = [my_index(G,i)[:10] for i in X]
index12 = [common_friends_number(G,i) for i in X]
index22 = [jaccard_index(G,i) for i in X]
index32 = [adamic_adar_index(G,i) for i in X]
index42 = [my_index(G,i) for i in X]
G.add_edges_from(new_edges, year = "2017-2018")

In [4]:
index1_result = [0 for i in range(len(index1))]
for i in range(len(index1)):
    for j in index1[i]:
        if G.has_edge(X[i],j):
            index1_result[i] += 1
common_user_result = sum(index1_result)/len(index1_result)
print("common_friend:",common_user_result)

common_friend: 1.183673469387755


In [5]:
index2_result = [0 for i in range(len(index2))]
for i in range(len(index2)):
    for j in index2[i]:
        if G.has_edge(j,X[i]):
            index2_result[i] += 1
print(len(index2_result))
jarr_result = sum(index2_result)/len(index2_result)
print("jaccard:",jarr_result)

49
jaccard: 0.7755102040816326


In [22]:
index3_result = [0 for i in range(len(index3))]
for i in range(len(index3)):
    for j in index3[i]:
        if G.has_edge(j,X[i]):
            index3_result[i] += 1
adam_result = sum(index3_result)/len(index3_result)
print("adam:", adam_result)

adam: 1.1020408163265305


In [19]:

index12_result = [0 for i in range(len(index12))]
for i in range(len(index12)):
    temp = []
    for j in G.neighbors(X[i]):
        if G[X[i]][j]['year'] == '2017-2018': 
            index = index12[i].index(j)
            temp.append(index)
    index12_result[i] = sum(temp)/len(list(G.neighbors(X[i])))
print("common_friend:", sum(index12_result)/len(index12_result))
        

common_friend: 1243.97823753638


In [18]:
index22_result = [0 for i in range(len(index22))]
for i in range(len(index22)):
    temp = []
    for j in G.neighbors(X[i]):
        if G[X[i]][j]['year'] == '2017-2018': 
            index = index22[i].index(j)
            temp.append(index)
    index22_result[i] = sum(temp)/len(list(G.neighbors(X[i])))
print("jaccard:",sum(index22_result)/len(index22_result))

jaccard: 1245.520980189047


In [17]:
index32_result = [0 for i in range(len(index32))]
for i in range(len(index32)):
    temp = []
    for j in G.neighbors(X[i]):
        if G[X[i]][j]['year'] == '2017-2018': 
            index = index32[i].index(j)
            temp.append(index)
    index32_result[i] = sum(temp)/len(list(G.neighbors(X[i])))
print("adam:",sum(index32_result)/len(index32_result))

adam: 1244.5469556501866


In [14]:
#Bonus: I think that for if a person want to make friends, he will make friends to whoever.
#Therefore, I do not think new link will be introduced only if people have common friends.
#So my thoughts is that if two people are both willing to make friends, they will know each 
#other even if they do not have common friends.
# My algorithm is the to introduce my index = length of neightbor A + length of neighbors B
# If they are the same willingness of making friends, probably they will be friends
# see function my_index() for implementation
# although my_index perform worse than other three in actual number, 
#but it perform much better in rank test

In [15]:
index4_result = [0 for i in range(len(index4))]
for i in range(len(index4)):
    for j in index4[i]:
        if G.has_edge(X[i],j):
            index4_result[i] += 1
my_index = sum(index4_result)/len(index4_result)
print("my_index:",my_index)

my_index: 0.1836734693877551


In [16]:
index42_result = [0 for i in range(len(index42))]
for i in range(len(index42)):
    temp = []
    for j in G.neighbors(X[i]):
        if G[X[i]][j]['year'] == '2017-2018': 
            index = index42[i].index(j)
            temp.append(index)
    index42_result[i] = sum(temp)/len(list(G.neighbors(X[i])))
print("my_index:",sum(index42_result)/len(index42_result))

my_index: 669.9326766483377
